### Notebook setup

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

### Create Engine/Session

In [2]:
import sqlalchemy as sa
import sqlalchemy.orm

engine = sa.create_engine(
    "cockroachdb://root@cockroach:26257/defaultdb?sslmode=disable", echo=True
)
engine

Engine(cockroachdb://root@cockroach:26257/defaultdb?sslmode=disable)

<IPython.core.display.Javascript object>

In [3]:
LocalSession = sa.orm.sessionmaker(engine)
LocalSession

sessionmaker(class_='Session', bind=Engine(cockroachdb://root@cockroach:26257/defaultdb?sslmode=disable), autoflush=True, autocommit=False, expire_on_commit=True)

<IPython.core.display.Javascript object>

### Drop/Create tables

In [4]:
from models import Base, User, Message

Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

2021-10-19 12:49:30,429 INFO sqlalchemy.engine.Engine select current_schema()
2021-10-19 12:49:30,430 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-10-19 12:49:30,434 INFO sqlalchemy.engine.Engine select version()
2021-10-19 12:49:30,435 INFO sqlalchemy.engine.Engine [generated in 0.00086s] {}
2021-10-19 12:49:30,512 INFO sqlalchemy.engine.Engine SELECT crdb_internal.increment_feature_counter(%(val)s)
2021-10-19 12:49:30,513 INFO sqlalchemy.engine.Engine [generated in 0.00096s] {'val': 'sqlalchemy-cockroachdb 1.4.1'}
2021-10-19 12:49:30,515 INFO sqlalchemy.engine.Engine SELECT crdb_internal.increment_feature_counter(%(val)s)
2021-10-19 12:49:30,516 INFO sqlalchemy.engine.Engine [cached since 0.003827s ago] {'val': 'sqlalchemy 1.4.25'}
2021-10-19 12:49:30,518 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-19 12:49:30,520 INFO sqlalchemy.engine.Engine SELECT table_name FROM information_schema.tables WHERE table_schema=%(schema)s
2021-10-19 12:49:30,521 INFO sqlalchemy.engine.En

<IPython.core.display.Javascript object>

### Add data

Nick, Eli, and Paul sing 99 bottles of beer...

In [5]:
users = [User(name="Nick"), User(name="Eli"), User(name="Paul")]
users

[User(id=None, name='Nick'),
 User(id=None, name='Eli'),
 User(id=None, name='Paul')]

<IPython.core.display.Javascript object>

In [6]:
with LocalSession() as session:
    session.add_all(users)
    session.commit()

2021-10-19 12:49:35,713 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-19 12:49:35,715 INFO sqlalchemy.engine.Engine INSERT INTO "user" (name) VALUES (%(name)s) RETURNING "user".id
2021-10-19 12:49:35,716 INFO sqlalchemy.engine.Engine [generated in 0.00113s] ({'name': 'Nick'}, {'name': 'Eli'}, {'name': 'Paul'})
2021-10-19 12:49:37,297 INFO sqlalchemy.engine.Engine COMMIT


<IPython.core.display.Javascript object>

In [7]:
import itertools

cycle = itertools.cycle(users)
messages = []

for i in reversed(range(100)):
    content = f"{i} bottles of beer on the wall..."
    user = next(cycle)
    message = Message(user=user, content=content)
    messages.append(message)

with LocalSession() as session:
    session.add_all(messages)
    session.commit()

2021-10-19 12:49:37,922 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-19 12:49:37,926 INFO sqlalchemy.engine.Engine SELECT "user".id AS user_id, "user".name AS user_name 
FROM "user" 
WHERE "user".id = %(pk_1)s
2021-10-19 12:49:37,928 INFO sqlalchemy.engine.Engine [generated in 0.00243s] {'pk_1': 703127149923827713}
2021-10-19 12:49:38,362 INFO sqlalchemy.engine.Engine SELECT "user".id AS user_id, "user".name AS user_name 
FROM "user" 
WHERE "user".id = %(pk_1)s
2021-10-19 12:49:38,363 INFO sqlalchemy.engine.Engine [cached since 0.4375s ago] {'pk_1': 703127149923794945}
2021-10-19 12:49:38,367 INFO sqlalchemy.engine.Engine SELECT "user".id AS user_id, "user".name AS user_name 
FROM "user" 
WHERE "user".id = %(pk_1)s
2021-10-19 12:49:38,369 INFO sqlalchemy.engine.Engine [cached since 0.4428s ago] {'pk_1': 703127149923237889}
2021-10-19 12:49:38,376 INFO sqlalchemy.engine.Engine INSERT INTO message (content, user_id) VALUES (%(content)s, %(user_id)s) RETURNING message.id
2021-10

<IPython.core.display.Javascript object>

### Query for data

#### Count query

In [8]:
### Expect to see 3 users here

with LocalSession() as session:
    statement = sa.select(sa.func.count(User.id))
    results = session.scalar(statement)

results

2021-10-19 12:49:40,347 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-19 12:49:40,350 INFO sqlalchemy.engine.Engine SELECT count("user".id) AS count_1 
FROM "user"
2021-10-19 12:49:40,351 INFO sqlalchemy.engine.Engine [generated in 0.00133s] {}
2021-10-19 12:49:40,353 INFO sqlalchemy.engine.Engine ROLLBACK


3

<IPython.core.display.Javascript object>

In [9]:
### Expect to see 100 messages here

with LocalSession() as session:
    statement = sa.select(sa.func.count(Message.id))
    results = session.scalar(statement)

results

2021-10-19 12:49:41,261 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-19 12:49:41,264 INFO sqlalchemy.engine.Engine SELECT count(message.id) AS count_1 
FROM message
2021-10-19 12:49:41,265 INFO sqlalchemy.engine.Engine [generated in 0.00107s] {}
2021-10-19 12:49:41,269 INFO sqlalchemy.engine.Engine ROLLBACK


100

<IPython.core.display.Javascript object>

#### Query users with messages

In [10]:
with LocalSession() as session:
    statement = sa.select(User).options(sa.orm.joinedload(User.messages))
    results = session.execute(statement)

results

2021-10-19 12:49:41,986 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-19 12:49:41,989 INFO sqlalchemy.engine.Engine SELECT "user".id, "user".name, message_1.id AS id_1, message_1.content, message_1.user_id 
FROM "user" LEFT OUTER JOIN message AS message_1 ON "user".id = message_1.user_id
2021-10-19 12:49:41,990 INFO sqlalchemy.engine.Engine [generated in 0.00125s] {}
2021-10-19 12:49:42,037 INFO sqlalchemy.engine.Engine ROLLBACK


<IPython.core.display.Javascript object>

In [11]:
users = results.unique().scalars().all()
users

[User(id=703127149923237889, name='Nick', messages=[Message(content='99 bottles of beer on the wall...', id=703127156151255041, user_id=703127149923237889), Message(content='0 bottles of beer on the wall...', id=703127156154630145, user_id=703127149923237889), Message(content='3 bottles of beer on the wall...', id=703127156154531841, user_id=703127149923237889), Message(content='6 bottles of beer on the wall...', id=703127156154433537, user_id=703127149923237889), Message(content='9 bottles of beer on the wall...', id=703127156154335233, user_id=703127149923237889), Message(content='12 bottles of beer on the wall...', id=703127156154236929, user_id=703127149923237889), Message(content='15 bottles of beer on the wall...', id=703127156154138625, user_id=703127149923237889), Message(content='18 bottles of beer on the wall...', id=703127156154040321, user_id=703127149923237889), Message(content='21 bottles of beer on the wall...', id=703127156153942017, user_id=703127149923237889), Message

<IPython.core.display.Javascript object>

In [12]:
users[0].name

'Nick'

<IPython.core.display.Javascript object>

In [13]:
users[0].messages[:5]

[Message(content='99 bottles of beer on the wall...', id=703127156151255041, user_id=703127149923237889),
 Message(content='0 bottles of beer on the wall...', id=703127156154630145, user_id=703127149923237889),
 Message(content='3 bottles of beer on the wall...', id=703127156154531841, user_id=703127149923237889),
 Message(content='6 bottles of beer on the wall...', id=703127156154433537, user_id=703127149923237889),
 Message(content='9 bottles of beer on the wall...', id=703127156154335233, user_id=703127149923237889)]

<IPython.core.display.Javascript object>

In [14]:
for m in users[0].messages[:5]:
    print(m.content)

99 bottles of beer on the wall...
0 bottles of beer on the wall...
3 bottles of beer on the wall...
6 bottles of beer on the wall...
9 bottles of beer on the wall...


<IPython.core.display.Javascript object>

#### Chained statement example

In [15]:
with LocalSession() as session:
    statement = sa.select(User)
    statement = statement.where(User.name == "Eli")
    statement = statement.options(sa.orm.selectinload(User.messages))
    results = session.execute(statement)

user = results.scalars().first()
user

2021-10-19 12:49:45,502 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-19 12:49:45,505 INFO sqlalchemy.engine.Engine SELECT "user".id, "user".name 
FROM "user" 
WHERE "user".name = %(name_1)s
2021-10-19 12:49:45,505 INFO sqlalchemy.engine.Engine [generated in 0.00076s] {'name_1': 'Eli'}
2021-10-19 12:49:45,508 INFO sqlalchemy.engine.Engine ROLLBACK
2021-10-19 12:49:45,511 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-19 12:49:45,513 INFO sqlalchemy.engine.Engine SELECT message.user_id AS message_user_id, message.id AS message_id, message.content AS message_content 
FROM message 
WHERE message.user_id IN (%(primary_keys_1)s)
2021-10-19 12:49:45,514 INFO sqlalchemy.engine.Engine [generated in 0.00148s] {'primary_keys_1': 703127149923794945}


User(id=703127149923794945, name='Eli', messages=[Message(content='98 bottles of beer on the wall...', id=703127156151418881, user_id=703127149923794945), Message(content='95 bottles of beer on the wall...', id=703127156151517185, user_id=703127149923794945), Message(content='92 bottles of beer on the wall...', id=703127156151615489, user_id=703127149923794945), Message(content='89 bottles of beer on the wall...', id=703127156151713793, user_id=703127149923794945), Message(content='86 bottles of beer on the wall...', id=703127156151812097, user_id=703127149923794945), Message(content='83 bottles of beer on the wall...', id=703127156151910401, user_id=703127149923794945), Message(content='80 bottles of beer on the wall...', id=703127156152008705, user_id=703127149923794945), Message(content='77 bottles of beer on the wall...', id=703127156152107009, user_id=703127149923794945), Message(content='74 bottles of beer on the wall...', id=703127156152205313, user_id=703127149923794945), Messa

<IPython.core.display.Javascript object>

In [16]:
user.name

'Eli'

<IPython.core.display.Javascript object>

In [17]:
for m in user.messages[:5]:
    print(m.content)

98 bottles of beer on the wall...
95 bottles of beer on the wall...
92 bottles of beer on the wall...
89 bottles of beer on the wall...
86 bottles of beer on the wall...


<IPython.core.display.Javascript object>

### Using classmethod queries

In [18]:
with LocalSession() as session:
    messages = Message.from_user(session, "Eli")

len(messages)

2021-10-19 12:49:47,405 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-19 12:49:47,407 INFO sqlalchemy.engine.Engine SELECT message.id, message.content, message.user_id 
FROM message JOIN "user" ON "user".id = message.user_id 
WHERE "user".name = %(name_1)s
2021-10-19 12:49:47,408 INFO sqlalchemy.engine.Engine [generated in 0.00092s] {'name_1': 'Eli'}
2021-10-19 12:49:47,414 INFO sqlalchemy.engine.Engine ROLLBACK


33

<IPython.core.display.Javascript object>

In [19]:
messages[:5]

[Message(content='98 bottles of beer on the wall...', id=703127156151418881, user_id=703127149923794945),
 Message(content='95 bottles of beer on the wall...', id=703127156151517185, user_id=703127149923794945),
 Message(content='92 bottles of beer on the wall...', id=703127156151615489, user_id=703127149923794945),
 Message(content='89 bottles of beer on the wall...', id=703127156151713793, user_id=703127149923794945),
 Message(content='86 bottles of beer on the wall...', id=703127156151812097, user_id=703127149923794945)]

<IPython.core.display.Javascript object>